In [1]:
from imdb import IMDb

# create an instance of the IMDb class
ia = IMDb()

# get a movie and print its director(s)
the_matrix = ia.get_movie('0133093')
for director in the_matrix['directors']:
    print(director['name'])

# show all information that are currently available for a movie
print(sorted(the_matrix.keys()))

# show all information sets that can be fetched for a movie
print(ia.get_movie_infoset())

# update a Movie object with more information
ia.update(the_matrix, ['technical'])
# show which keys were added by the information set
print(the_matrix.infoset2keys['technical'])
# print one of the new keys
print(the_matrix.get('tech'))


Lana Wachowski
Lilly Wachowski
['akas', 'animation department', 'art department', 'art directors', 'aspect ratio', 'assistant directors', 'box office', 'camera department', 'canonical title', 'cast', 'casting department', 'casting directors', 'certificates', 'cinematographers', 'color info', 'composers', 'costume departmen', 'costume designers', 'countries', 'country codes', 'cover url', 'director', 'directors', 'distributors', 'editorial department', 'editors', 'full-size cover url', 'genres', 'kind', 'language codes', 'languages', 'location management', 'long imdb canonical title', 'long imdb title', 'make up department', 'miscellaneous', 'music department', 'original air date', 'other companies', 'plot', 'plot outline', 'producers', 'production companies', 'production designers', 'production managers', 'rating', 'runtimes', 'script department', 'set decorators', 'smart canonical title', 'smart long imdb canonical title', 'sound department', 'sound mix', 'special effects', 'special e

In [3]:
import pandas as pd
import numpy as np
import re
import lxml

from bs4 import BeautifulSoup
from requests import get
%matplotlib inline

In [7]:
url= "https://www.imdb.com/search/title/?count=100&title_type=feature,tv_series&ref_=nv_wl_img_2"

In [8]:
page = get(url)

In [9]:

soup = BeautifulSoup(page.content, 'lxml')

In [10]:

content = soup.find(id="main")

In [17]:
articleTitle = soup.find("h1", class_="header").text.replace("\n","")

movieFrame = content.find_all("div", class_="lister-item mode-advanced")

movieFirstLine = movieFrame[0].find("h3", class_="lister-item-header")

movieTitle = movieFirstLine.find("a").text
 
movieDate = re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text)
 
movieRunTime = movieFrame[0].find("span", class_="runtime").text[:-4]
 
movieGenre = movieFrame[0].find("span", class_="genre").text.rstrip().replace("\n","").split(",")
 
movieRating = movieFrame[0].find("strong").text

movieCast = movieFrame[0].find("p", class_="")

movieDesc = movieFrame[0].find_all("p", class_="text-muted")[-1].text.lstrip()

In [18]:
movieScore = movieFrame[0].find("span", class_="metascore unfavorable").text.rstrip()
 

 


AttributeError: 'NoneType' object has no attribute 'text'

In [19]:

#Movie Director and Movie Stars
try:
    casts = movieCast.text.replace("\n","").split('|')
    casts = [x.strip() for x in casts]
    casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    movieDirector = casts[0]
    movieStars = [x.strip() for x in casts[1].split(",")]
except:
    casts = movieCast.text.replace("\n","").strip()
    movieDirector = np.nan
    movieStars = [x.strip() for x in casts.split(",")]

In [20]:

movieNumbers = movieFrame[0].find_all("span", attrs={"name": "nv"})

In [21]:
if len(movieNumbers) == 2:
    movieVotes = movieNumbers[0].text
    movieGross = movieNumbers[1].text
else:
    movieVotes = movieNumbers[0].text
    movieGross = np.nan


In [30]:

import lxml
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from requests import get

url1 = "https://www.imdb.com/search/title/?count=100&title_type=feature,tv_series&ref_=nv_wl_img_2"

class IMDB(object):
    """docstring for IMDB"""
    def __init__(self, url):
        super(IMDB, self).__init__()
        page = get(url)

        self.soup = BeautifulSoup(page.content, 'lxml')

    def articleTitle(self):
        return self.soup.find("h1", class_="header").text.replace("\n","")

    def bodyContent(self):
        content = self.soup.find(id="main")
        return content.find_all("div", class_="lister-item mode-advanced")

    def movieData(self):
        movieFrame = self.bodyContent()
        movieTitle = []
        movieDate = []
        movieRunTime = []
        movieGenre = []
        movieRating = []
        movieScore = []
        movieDescription = []
        movieDirector = []
        movieStars = []
        movieVotes = []
        movieGross = []
        for movie in movieFrame:
            movieFirstLine = movie.find("h3", class_="lister-item-header")
            movieTitle.append(movieFirstLine.find("a").text)
            movieDate.append(re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text))
            try:
                movieRunTime.append(movie.find("span", class_="runtime").text[:-4])
            except:
                movieRunTime.append(np.nan)
            movieGenre.append(movie.find("span", class_="genre").text.rstrip().replace("\n","").split(","))
            try:
                movieRating.append(movie.find("strong").text)
            except:
                movieRating.append(np.nan)
            try:
                movieScore.append(movie.find("span", class_="metascore unfavorable").text.rstrip())
            except:
                movieScore.append(np.nan)
            movieDescription.append(movie.find_all("p", class_="text-muted")[-1].text.lstrip())
            movieCast = movie.find("p", class_="")

            try:
                casts = movieCast.text.replace("\n","").split('|')
                casts = [x.strip() for x in casts]
                casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
                movieDirector.append(casts[0])
                movieStars.append([x.strip() for x in casts[1].split(",")])
            except:
                casts = movieCast.text.replace("\n","").strip()
                movieDirector.append(np.nan)
                movieStars.append([x.strip() for x in casts.split(",")])

            movieNumbers = movie.find_all("span", attrs={"name": "nv"})

            if len(movieNumbers) == 2:
                movieVotes.append(movieNumbers[0].text)
                movieGross.append(movieNumbers[1].text)
            elif len(movieNumbers) == 1:
                movieVotes.append(movieNumbers[0].text)
                movieGross.append(np.nan)
            else:
                movieVotes.append(np.nan)
                movieGross.append(np.nan)

        movieData = [movieTitle, movieDate, movieRunTime, movieGenre, movieRating, movieScore, movieDescription,
                            movieDirector, movieStars, movieVotes, movieGross]
        return movieData

In [31]:

if __name__ == '__main__':
    site1 = IMDB(url1)
    print("Subject: ", site1.articleTitle())
    data = site1.movieData()
    for i in range(len(data)):
        print(data[i][:]) #Print the data

Subject:  Feature Film/TV Series(Sorted by Popularity Ascending) 
['The Mandalorian', 'The Irishman', 'Marriage Story', 'Wonder Woman 1984', '6 Underground', 'Ghostbusters: Afterlife', 'Knives Out', 'Star Wars: Episode IX - The Rise of Skywalker', 'Jumanji: The Next Level', 'V-Wars', 'The Marvelous Mrs. Maisel', 'Watchmen', 'Once Upon a Time... in Hollywood', 'Virgin River', 'Free Guy', 'The Witcher', 'His Dark Materials', 'Vikings', 'The Crown', 'Frozen 2', 'Rick and Morty', 'Joker', 'Ad Astra', 'See', 'Bombshell', 'Arrow', 'Richard Jewell', 'Mr. Robot', 'Doctor Sleep', 'Reprisal', 'Cats', 'No Time to Die', 'Gisaengchung', 'Little Women', 'The Expanse', 'The Flash', 'Supergirl', 'Hustlers', 'Ford v. Ferrari', 'Game of Thrones', 'Black Christmas', '1917', 'Batwoman', 'Silicon Valley', 'Home Alone', 'Code 8', 'Peaky Blinders', 'Uncut Gems', 'In the Heights', 'Love Actually', 'Klaus', 'The Morning Show', 'Supernatural', 'The Walking Dead', 'The Boys', "National Lampoon's Christmas Vacati